In [ ]:
!pip install -r requirements.txt

In [ ]:
from pypots.imputation import SAITS
import pandas as pd
import numpy as np

In [2]:
def form_train(array, seq_len, step):
    result_list = []
    for i in range(seq_len, len(array) + 1, step):
        result_list.append(np.expand_dims(array[i - seq_len : i], axis=0))
    return np.concatenate(result_list, axis=0)

In [ ]:
data = pd.read_csv("data.csv")
seq_len = 40
train_values = form_train(data.drop("Date", axis=1).values, seq_len, 1)
assert np.allclose(
    train_values[1][0][~np.isnan(train_values[1][0])], 
    train_values[0][1][~np.isnan(train_values[0][1])]
)

train_data = {"X": train_values, "missing_mask": np.zeros_like(train_values)}

In [ ]:
model_params = {
    "n_layers": 2,
    "d_model": 64,
    "d_inner": 32,
    "n_heads": 4,
    "d_k": 32,
    "d_v": 32,
    "dropout": 0.1,
    "epochs": 20,
    "n_steps": seq_len,
    "n_features": train_data["X"].shape[-1],
    "epochs": 20
}
model = SAITS(**model_params)
# model.fit(train_data)
# model.save_model("weights", "pretrained_weights.pypots")
model.load_model("weights/pretrained_weights.pypots")

In [ ]:
input_data_t = {'X': torch.tensor(train_data['X'], dtype=torch.float32),
               'missing_mask': torch.tensor(train_data['missing_mask'], dtype=torch.float32)}

In [ ]:
out = model.model(input_data_t, True, False)